# **🏠 재개발**
> 재개발 지역이거나 재개발이 이루어질 예정인 집은 값이 오른다.
> 이러한 특성을 활용해 구별 재개발 지역 개수를 count

## Contents
- Library Import
- Data Load

## 1. Library Import
- 필요한 라이브러리를 불러옵니다.

In [15]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import re


## 2. Data Load

In [68]:
# 데이터 불러오기
rebuild = pd.read_csv('../data/주택재개발사업_추진실적.csv')

In [70]:
# 필요없는 변수 제거
r = rebuild.drop(['자치구별(1)', '항목'], axis = 1)[rebuild['자치구별(2)'] != '소계'].reset_index()

In [71]:
r['데이터'][r['데이터'] == '-'] = 0
r['데이터'] = r['데이터'].astype('int64')

In [74]:
r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   index    800 non-null    int64 
 1   자치구별(2)  800 non-null    object
 2   시점       800 non-null    int64 
 3   실적별(1)   800 non-null    object
 4   데이터      800 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 31.4+ KB


- 데이터가 2022년까지 밖에 없어서 2022년에 시행중인 값을 2023년에 반영하기로 결정

In [79]:
# 자치구별, 연도별 재개발개수
newre = r.groupby(['자치구별(2)', '시점']).sum().reset_index()[['자치구별(2)', '시점', '데이터']]
newre.columns = ['구', '연도', '재개발개수']

In [80]:
# 2022에서 시행중인 것만 뽑아서 2023년에 반영
r2023 = r[(r['시점'] == 2022) & (r['실적별(1)'] == '시행중')][['자치구별(2)', '시점', '데이터']]

# 시점 2023대입 (시행중)
r2023['시점'] = 2023
r2023.columns = ['구', '연도', '재개발개수']

In [86]:
# 2007 ~ 2023
k = pd.concat([newre, r2023])

In [87]:
# 저장
k.to_csv('../data/재개발개수.csv', index=False)